In [ ]:
# Load stock market data
data = pd.read_csv('Stock_Market.csv')

# Normalize data
hutchison = data['Hutchison'].values
sun_hung_kai_prop = data['Sun_Hung_Kai_Prop'].values
cheung_kong = data['Cheung_Kong'].values

X = (hutchison - np.min(hutchison)) / (np.max(hutchison) - np.min(hutchison))
Y = (sun_hung_kai_prop - np.min(sun_hung_kai_prop)) / (np.max(sun_hung_kai_prop) - np.min(sun_hung_kai_prop))
U = (cheung_kong - np.min(cheung_kong)) / (np.max(cheung_kong) - np.min(cheung_kong))

# X -> Y model for both models
traindata1_x_to_y = np.array([X, Y, U]).transpose()
traindata1_y_to_x = np.array([Y, X, U]).transpose()

# Run CANM model
result1_canm_x_to_y = fit(traindata1_x_to_y, model_type='canm', verbose=True)
result1_canm_y_to_x = fit(traindata1_y_to_x, model_type='canm', N=1, verbose=True)

# Run TransformerVAE model
result1_transformer_x_to_y = fit(traindata1_x_to_y, model_type='transformer', verbose=True)
result1_transformer_y_to_x = fit(traindata1_y_to_x, model_type='transformer', N=1, verbose=True)

# Compare results for CANM
print("CCANM Results:")
print(f"X -> Y Train Likelihood: {result1_canm_x_to_y['train_likelihood']}")
print(f"Y -> X Train Likelihood: {result1_canm_y_to_x['train_likelihood']}")
print(f"X -> Y Mean Train Score: {np.mean(result1_canm_x_to_y['train_score'])}")
print(f"Y -> X Mean Train Score: {np.mean(result1_canm_y_to_x['train_score'])}")

# Compare results for TransformerVAE
print("\nTV-CCANM Results:")
print(f"X -> Y Train Likelihood: {result1_transformer_x_to_y['train_likelihood']}")
print(f"Y -> X Train Likelihood: {result1_transformer_y_to_x['train_likelihood']}")
print(f"X -> Y Mean Train Score: {np.mean(result1_transformer_x_to_y['train_score'])}")
print(f"Y -> X Mean Train Score: {np.mean(result1_transformer_y_to_x['train_score'])}")

# Determine causal direction based on train likelihood
if result1_canm_x_to_y['train_likelihood'] > result1_canm_y_to_x['train_likelihood']:
    print("\nCCANM suggests causal direction based on train likelihood: X -> Y")
else:
    print("\nCCANM suggests causal direction based on train likelihood: Y -> X")

if result1_transformer_x_to_y['train_likelihood'] > result1_transformer_y_to_x['train_likelihood']:
    print("TV-CCANM suggests causal direction based on train likelihood: X -> Y")
else:
    print("TV-CCANM suggests causal direction based on train likelihood: Y -> X")

# Determine causal direction based on mean train score
if np.mean(result1_canm_x_to_y['train_score']) < np.mean(result1_canm_y_to_x['train_score']):
    print("CCANM suggests causal direction based on mean train score: X -> Y")
else:
    print("CCANM suggests causal direction based on mean train score: Y -> X")

if np.mean(result1_transformer_x_to_y['train_score']) < np.mean(result1_transformer_y_to_x['train_score']):
    print("TV-CCANM suggests causal direction based on mean train score: X -> Y")
else:
    print("TV-CCANM suggests causal direction based on mean train score: Y -> X")

# Individual Train Score Graphs for Each Model

# CANM Model
plt.figure(figsize=(8, 5))
plt.plot(result1_canm_x_to_y['train_score'], label="CCANM Hutchison -> Sun_Hung_Kai_Prop", linestyle='--')
plt.plot(result1_canm_y_to_x['train_score'], label="CCANM Sun_Hung_Kai_Prop -> Hutchison", linestyle=':')
plt.xlabel("Epochs")
plt.ylabel("Negative Train Score")
plt.title("Negative Train Scores for CCANM")
plt.legend()
plt.show()

# TransformerVAE Model
plt.figure(figsize=(8, 5))
plt.plot(result1_transformer_x_to_y['train_score'], label="TV-CCANM Hutchison -> Sun_Hung_Kai_Prop", linestyle='-')
plt.plot(result1_transformer_y_to_x['train_score'], label="TV-CCANM Sun_Hung_Kai_Prop -> Hutchison", linestyle='-.')
plt.xlabel("Epochs")
plt.ylabel("Negative Train Score")
plt.title("Negative Train Scores for TV-CCANM")
plt.legend()
plt.show()
